In [ ]:
!spades.py --only-error-correction --pe1-1 data/4/ecoli_400K_err_1.fastq.gz --pe1-2 data/4/ecoli_400K_err_2.fastq.gz -o data/4/corrected

In [3]:
!bwa index data/4/MG1655-K12.first400K.fasta

[bwa_index] Pack FASTA... 0.01 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.09 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.03 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index data/4/MG1655-K12.first400K.fasta
[main] Real time: 0.144 sec; CPU: 0.151 sec


### Corrected reads

In [ ]:
%%capture
!bwa mem data/4/MG1655-K12.first400K.fasta data/4/ecoli_400K_err_1.fastq.00.0_0.cor.fastq.gz data/4/ecoli_400K_err_2.fastq.00.0_0.cor.fastq.gz > data/4/corrected_alignment.sam

In [8]:
!samtools flagstat data/4/corrected_alignment.sam

2716054 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2715944 + 0 mapped (100.00% : N/A)
2716054 + 0 paired in sequencing
1358027 + 0 read1
1358027 + 0 read2
2714586 + 0 properly paired (99.95% : N/A)
2715876 + 0 with itself and mate mapped
68 + 0 singletons (0.00% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


### Original reads

In [ ]:
%%capture
!bwa mem data/4/MG1655-K12.first400K.fasta data/4/ecoli_400K_err_1.fastq.gz data/4/ecoli_400K_err_2.fastq.gz > data/4/original_alignment.sam

In [10]:
!samtools flagstat data/4/original_alignment.sam

2763204 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2762617 + 0 mapped (99.98% : N/A)
2763204 + 0 paired in sequencing
1381602 + 0 read1
1381602 + 0 read2
2760820 + 0 properly paired (99.91% : N/A)
2762132 + 0 with itself and mate mapped
485 + 0 singletons (0.02% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [11]:
import pysam

import numpy as np
import pandas as pd

In [12]:
def get_dict_from_reference_pos_to_read_nuleotide(read):
    read_seq = read.query_sequence
    pairs = read.get_aligned_pairs(matches_only=False)
    
    return dict([(ref_pos, read_seq[read_pos] if read_pos is not None else None) for read_pos, ref_pos in pairs])
    
def get_dict_from_reference_pos_to_reference_nucleotide(read):
    pairs = read.get_aligned_pairs(matches_only=False, with_seq=True)
    
    return dict([(ref_pos, ref_n) for read_pos, ref_pos, ref_n in pairs])

def count_corrections(corr_sam_file, orig_sam_file):
    corr_sam = pysam.AlignmentFile(corr_sam_file, "r")
    orig_sam = pysam.AlignmentFile(orig_sam_file, "r")
    
    ans = np.zeros((2,3), dtype=int)
    
    while True:
        try:
            corr_read, orig_read = next(corr_sam), next(orig_sam)
            
            if corr_read.is_unmapped or corr_read.is_supplementary: continue
            if orig_read.is_unmapped or orig_read.is_supplementary: continue
            
            # count those reads as absent?
            while corr_read.query_name != orig_read.query_name:
                orig_read = next(orig_sam)
                
            ref_positions_orig = orig_read.get_reference_positions()
            ref_positions_corr = corr_read.get_reference_positions()   
            
            mapped_orig = get_dict_from_reference_pos_to_read_nuleotide(orig_read)
            mapped_corr = get_dict_from_reference_pos_to_read_nuleotide(corr_read)
            
            ref_pos_to_nucleotide = get_dict_from_reference_pos_to_reference_nucleotide(orig_read)
            
            for ref_pos in set(mapped_orig.keys()) & set(mapped_corr.keys()):
                ref_base = ref_pos_to_nucleotide[ref_pos]
                orig_base = mapped_orig[ref_pos]
                corr_base = mapped_corr[ref_pos]
                
                if corr_base == 'N':
                    ans[int(orig_base == ref_base)][2] += 1
                    continue
                        
                if orig_base != ref_base:
                    ans[0][int(corr_base == ref_base)] += 1
                else:
                    ans[1][int(corr_base == orig_base)] += 1   
        
        except StopIteration:
            break
            
    return ans

arr = count_corrections('data/4/corrected_alignment.sam', 'data/4/original_alignment.sam')

In [13]:
def make_ans_human_readable(ans):
    ans_str = np.array(ans, dtype=str)
    ans_annotation = np.array([['Undetected error: ', 'Detected & corrected error: ', 'Detected and removed error: '], 
                               ['Falsely corrected error: ', 'Correctly unmodified base: ', 'Incorrectly removed base: ']])

    ans_concated = np.core.defchararray.add(ans_annotation, ans_str)
    df = pd.DataFrame(ans_concated, columns=["Error in corrected reads", "Correct base in corrected reads", "Base is absent in corrected reads"])
    df.set_index([['Error in raw data', 'Correct base in raw data']], inplace=True)

    return df

df = make_ans_human_readable(arr)
df

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,Undetected error: 1058063,Detected & corrected error: 182,Detected and removed error: 326925
Correct base in raw data,Falsely corrected error: 545,Correctly unmodified base: 246936100,Incorrectly removed base: 380383


### Краткое изложение метода
Выравниваем исходные и скорретированые риды на референс и идём по обоим выравниваниям сразу сразу итератором.
Попутно скипая выкинутые риды, которых нет в скорректированных (их я считать как absent не стал).
Когда получили два рида, непонятно насколько одинаково и хорошо они выравнились, поэтому чтобы не писать сосисочный код делаем следующие: <br>
1) Для каждого рида строим словарь типа 'позиция в референсе' -> 'нуклеотид в риде' <br>
2) Строим такой же словарь для рефенса, 'позиция в референсе' -> 'нуклеотид в референсе' <br>
3) Пересекаем множество ключей словарей и в цикле проходимся по всем тройкам баз <br>
Получив соответствующие базы, уже достаточно написать несколько ифоф, чтобы посчитать ответ.